In [2]:
import torch
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transform

In [3]:
class SismicDataset(Dataset):
    def __init__(self, max_length=572427):
        # Cargar el catálogo CSV
        self.cat = pd.read_csv('./data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv')
        self.data_directory = './data/lunar/training/data/S12_GradeA'
        self.max_length = max_length  # Define la longitud máxima de las secuencias

    def __len__(self):
        return len(self.cat)

    def __getitem__(self, idx):
        # Obtener los datos de la fila correspondiente
        row = self.cat.iloc[idx]
        test_filename = row['filename']
        arrival_time_rel = row['time_rel(sec)']

        # Construir la ruta al archivo .mseed
        mseed_file = os.path.join(self.data_directory, f'{test_filename}.mseed')

        # Leer los datos de la señal usando obspy
        st = read(mseed_file)
        tr = st.traces[0]
        tr_data = tr.data.astype(np.float32)  # Convertir los datos a float32

        # Verificar si la señal es más larga que la longitud máxima y truncarla si es necesario
        if len(tr_data) > self.max_length:
            tr_data = tr_data[:self.max_length]
        # Si la señal es más corta, añadir ceros al final (padding)
        elif len(tr_data) < self.max_length:
            tr_data = np.pad(tr_data, (0, self.max_length - len(tr_data)), 'constant')

        # Convertir los datos a tensores
        signal = torch.tensor(tr_data, dtype=torch.float32)
        label = torch.tensor(arrival_time_rel, dtype=torch.float32)

        return signal, label

In [4]:
class SeismicCNN(nn.Module):
    def __init__(self):
        super(SeismicCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        self.conv2 = nn.Conv1d(16, 32, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.dropout = nn.Dropout(0.5)

        # Ahora podemos definir fc1 con una dimensión de entrada fija
        # Cálculo dinámico de la dimensión de la salida de las capas convolucionales

        self._to_linear = None ################################### 
        self.fc1 = None  # Se inicializa dinámicamente más adelante

        self.relu_fc1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 64)
        self.relu_fc2 = nn.ReLU()
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        # x tiene forma (batch_size, signal_length)
        x = x.unsqueeze(1)  # Añadir dimensión de canal: (batch_size, 1, signal_length)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.dropout(x)
            ############################### Flaten
        if self._to_linear is None:
            self._to_linear = x.view(x.size(0), -1).size(1)
            #print(f"Output size after convolutional layers: {self._to_linear}")
            # Inicializar la primera capa lineal con la dimensión correcta
            self.fc1 = nn.Linear(self._to_linear, 64)
            self.fc1.to(x.device)

        #print('before :', x.shape)
        x = x.view(x.size(0), -1)  # Aplanar para la capa totalmente conectada
        #print('after:', x.shape)
        x = self.fc1(x)
        x = self.relu_fc1(x)
        x = self.fc2(x)
        x = self.relu_fc2(x)
        x = self.fc4(x)
        return x.squeeze()  # Devolver tensor de forma (batch_size)

In [5]:
#Extraer datos
dataset = SismicDataset()
#Cargar datos
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=0)

#modelo
model = SeismicCNN()

#training loop
total_samples = len(dataset)
n_iter =  int(total_samples/4)
print(total_samples, n_iter)

######################
#parameters
learning_rate=0.001
n_epochs = 5

# Definir funcion de pérdida y optimizador
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


# Mover el modelo al dispositivo adecuado
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Listas para almacenar las pérdidas
# train_losses = []
#val_losses = []

for epoch in range(n_epochs):
    for i, (inputs, labels) in  enumerate(dataloader):
        #origin shape : [4, 572427]
        #forward, backward, update
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(inputs.shape, labels.shape)
        #forward
        outputs = model(inputs)
        print('Outputs: ', outputs)
        train_loss = loss(labels, outputs)

        #backward
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()       

        if (i+1)%10==0:
            print(f'epoch {epoch+1}/{n_epochs}, step {i+1}/{n_iter}, inputs {inputs.shape}, Training Loss: {train_loss.item():.4f}')


76 19
Outputs:  tensor([-0.0406, -0.0391, -0.0412, -0.0404], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Outputs:  tensor([17.5346, 17.4731, 17.7028, 17.4958], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Outputs:  tensor([42370284., 44751920., 55967604., 54758308.], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Outputs:  tensor([-1.3927e+26, -1.3074e+26, -2.0303e+26, -9.6211e+25], device='cuda:0',
       grad_fn=<SqueezeBackward0>)
Outputs:  tensor([nan, nan, nan, nan], device='cuda:0', grad_fn=<SqueezeBackward0>)
Outputs:  tensor([nan, nan, nan, nan], device='cuda:0', grad_fn=<SqueezeBackward0>)
Outputs:  tensor([nan, nan, nan, nan], device='cuda:0', grad_fn=<SqueezeBackward0>)
Outputs:  tensor([nan, nan, nan, nan], device='cuda:0', grad_fn=<SqueezeBackward0>)
Outputs:  tensor([nan, nan, nan, nan], device='cuda:0', grad_fn=<SqueezeBackward0>)


KeyboardInterrupt: 

In [ ]:
transform = transform.Compose(
    [transforms.ToTensor(),
     transforms.Normalize()
    ]
)